## add patch

```python
def add_patch(self, p):
    """
    Add a `~.Patch` to the axes' patches; return the patch.
    """
    self._set_artist_props(p)          # _01: 这个函数，研究
    if p.get_clip_path() is None:
        p.set_clip_path(self.patch)    # _02: self.patch 是
    self._update_patch_limits(p)       # _03: _update_patch_limits
    self.patches.append(p)
    p._remove_method = self.patches.remove
    return p
```

01. 设置 figure, axes, transform

```python
def _set_artist_props(self, a):
    """Set the boilerplate props for artists added to axes."""
    a.set_figure(self.figure)
    if not a.is_transform_set():
        a.set_transform(self.transData)

    a.axes = self
    if a.mouseover:
        self._mouseover_set.add(a)
```

02. self.patch = self._gen_axes_patch()

```python
def _gen_axes_patch(self):
    """
    Returns
    -------
    Patch
        The patch used to draw the background of the axes.  It is also used
        as the clipping path for any data elements on the axes.

        In the standard axes, this is a rectangle, but in other projections
        it may not be.

    Notes
    -----
    Intended to be overridden by new projection types.
    """
    return mpatches.Rectangle((0.0, 0.0), 1.0, 1.0)
```

03. transform 这一段不理解

```python
def _update_patch_limits(self, patch):
    """Update the data limits for the given patch."""
    # hist can add zero height Rectangles, which is useful to keep
    # the bins, counts and patches lined up, but it throws off log
    # scaling.  We'll ignore rects with zero height or width in
    # the auto-scaling

    # cannot check for '==0' since unitized data may not compare to zero
    # issue #2150 - we update the limits if patch has non zero width
    # or height.
    if (isinstance(patch, mpatches.Rectangle) and
            ((not patch.get_width()) and (not patch.get_height()))):
        return
    vertices = patch.get_path().vertices
    if vertices.size > 0:
        xys = patch.get_patch_transform().transform(vertices)
        if patch.get_data_transform() != self.transData:
            patch_to_data = (patch.get_data_transform() -
                             self.transData)
            xys = patch_to_data.transform(xys)

        updatex, updatey = patch.get_transform().\
            contains_branch_seperately(self.transData)
        self.update_datalim(xys, updatex=updatex,
                            updatey=updatey)
```

## add collection

```python
def add_collection(self, collection, autolim=True):
    """
    Add a `~.Collection` to the axes' collections; return the collection.
    """
    label = collection.get_label()
    if not label:
        collection.set_label('_collection%d' % len(self.collections))
    self.collections.append(collection)
    collection._remove_method = self.collections.remove
    self._set_artist_props(collection)

    if collection.get_clip_path() is None:
        collection.set_clip_path(self.patch)

    if autolim:
        # Make sure viewLim is not stale (mostly to match
        # pre-lazy-autoscale behavior, which is not really better).
        self._unstale_viewLim()
        self.update_datalim(collection.get_datalim(self.transData))

    self.stale = True
    return collection
```

add_patch与add_collection很相似，差异有：
1. self.patches.append 与 self.collections.append

不能理解的是，path通过PathCollections及add_collections只是存入了Axes.collections而已，它是如何绘制的？不需要转换成patch吗？

In [1]:
import matplotlib.pyplot as plt
import matplotlib.path as mpath

In [ ]:
mpath.Path.u